In [149]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [151]:
df_tax=pd.read_excel('filtered_file_tax.xlsx')

In [153]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97 entries, 0 to 96
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   C+A1:U1ase Id        97 non-null     object
 1   Case Title           97 non-null     object
 2   Court Type           96 non-null     object
 3   Jurisdiction         95 non-null     object
 4   Case Year            97 non-null     int64 
 5   Category of case     97 non-null     object
 6   Case keywords        97 non-null     object
 7   Number of evidences  97 non-null     object
 8   Outcome in favor of  94 non-null     object
 9   Outcome in Court     95 non-null     object
 10  Main Issue           94 non-null     object
 11  Tax                  97 non-null     int64 
 12  Outcome              97 non-null     int64 
dtypes: int64(3), object(10)
memory usage: 10.0+ KB


In [177]:
df_tax.head()

,C+A1:U1ase Id,Case Title,Court Type,Jurisdiction,Case Year,Category of case,Case keywords,Number of evidences,Outcome in favor of,Outcome in Court,Main Issue,Tax,Outcome
0,C01,Vishnu Agencies (Pvt.) Ltd. Etc vs Commercial ...,Vishnu Agencies (Pvt.) Ltd. Etc vs Commercial ...,Supreme Court of India,1977,Tax Law,sales tax compulsory sale essential commoditie...,Medium,Respondent,Appeal dismissed,Taxability of compulsory sales under statutory...,1,0
1,C02,Vikas Sales Corporation And Anr. Etc. ... vs C...,Civil Appellate Jurisdiction,Supreme Court of India,1996,Tax Law,sales tax rep licence exim scrip import export...,Medium,Respondent,Appeal dismissed,Taxability of REP Licences/Exim Scrips,1,0
2,C04,"Commercial Tax Officer, Rajasthan vs Binani Ce...",Civil Appellate Jurisdiction,Supreme Court of India,2014,Tax Law,sales tax tax exemption industrial unit fci ra...,Low,Against Respondent,Appeal allowed,Eligibility for tax exemption under the Rajast...,1,1
3,C09,Ald Automotive Pvt Ltd vs The Commercial Tax O...,Supreme Court,India,2018,"Tax Law, Constitutional Law",vat input tax credit statutory interpretation,Medium,Respondent,Upheld the validity of Section 19(11) and deni...,Challenge to Section 19(11) of Tamil Nadu Valu...,1,0
4,C11,State Of Tamil Nadu And Another vs Board Of Tr...,Supreme Court of India,India,1999,"Tax Law, Constitutional Law",port trust sales tax statutory duty business a...,Low,Respondent,Judgment in favor of the Port Trust,Whether the Port Trust is a 'dealer' under the...,1,0


In [179]:
df.isnull().sum()

C+A1:U1ase Id          0
Case Title             0
Court Type             1
Jurisdiction           2
Case Year              0
Category of case       0
Case keywords          0
Number of evidences    0
Outcome in favor of    3
Outcome in Court       2
Main Issue             3
Tax                    0
Outcome                0
dtype: int64

In [181]:
df.Tax.value_counts()

Tax
1    97
Name: count, dtype: int64

In [183]:
df.Outcome .value_counts()

Outcome
0    59
1    38
Name: count, dtype: int64

In [185]:
import re 
import string

In [187]:
alphanumeric=lambda x:re.sub('\w*\d\w*','',x)
pun_lower=lambda x:re.sub('[%s]' % re.escape(string.punctuation),'',x.lower())
remove_n=lambda x:re.sub('\n','',x)
remove_non_ascii=lambda x:re.sub(r'[^\x00-\x7f]',r'',x)
df_tax['Case keywords']=df_tax['Case keywords'].map(alphanumeric).map(pun_lower).map(remove_n).map(remove_non_ascii)

In [189]:
from sklearn.model_selection import train_test_split,KFold,cross_val_score
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [191]:
df_tax.head()

,C+A1:U1ase Id,Case Title,Court Type,Jurisdiction,Case Year,Category of case,Case keywords,Number of evidences,Outcome in favor of,Outcome in Court,Main Issue,Tax,Outcome
0,C01,Vishnu Agencies (Pvt.) Ltd. Etc vs Commercial ...,Vishnu Agencies (Pvt.) Ltd. Etc vs Commercial ...,Supreme Court of India,1977,Tax Law,sales tax compulsory sale essential commoditie...,Medium,Respondent,Appeal dismissed,Taxability of compulsory sales under statutory...,1,0
1,C02,Vikas Sales Corporation And Anr. Etc. ... vs C...,Civil Appellate Jurisdiction,Supreme Court of India,1996,Tax Law,sales tax rep licence exim scrip import export...,Medium,Respondent,Appeal dismissed,Taxability of REP Licences/Exim Scrips,1,0
2,C04,"Commercial Tax Officer, Rajasthan vs Binani Ce...",Civil Appellate Jurisdiction,Supreme Court of India,2014,Tax Law,sales tax tax exemption industrial unit fci ra...,Low,Against Respondent,Appeal allowed,Eligibility for tax exemption under the Rajast...,1,1
3,C09,Ald Automotive Pvt Ltd vs The Commercial Tax O...,Supreme Court,India,2018,"Tax Law, Constitutional Law",vat input tax credit statutory interpretation,Medium,Respondent,Upheld the validity of Section 19(11) and deni...,Challenge to Section 19(11) of Tamil Nadu Valu...,1,0
4,C11,State Of Tamil Nadu And Another vs Board Of Tr...,Supreme Court of India,India,1999,"Tax Law, Constitutional Law",port trust sales tax statutory duty business a...,Low,Respondent,Judgment in favor of the Port Trust,Whether the Port Trust is a 'dealer' under the...,1,0


In [252]:
def model(dataf,lbl):
    # split the data
    x=dataf['Case keywords']+dataf['Number of evidences']
    y=dataf[lbl]

    # spliting into train and test
    x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=50)

    v=CountVectorizer()

    x_train_v=v.fit_transform(x_train.values)
    x_test_v=v.transform(x_test.values)

    # machine learning models
    lr=LogisticRegression()
    lr.fit(x_train_v,y_train)

    mnb=MultinomialNB()
    mnb.fit(x_train_v,y_train)

    svm=SVC()
    svm.fit(x_train_v,y_train)

    rfc=RandomForestClassifier()
    rfc.fit(x_train_v,y_train)

    # score
    f1_score_data={ 'F1 Score':[ f1_score(lr.predict(x_test_v),y_test),f1_score(mnb.predict(x_test_v),y_test),
                           f1_score(svm.predict(x_test_v),y_test),f1_score(rfc.predict(x_test_v),y_test)]}
    
    # saving scores in dataframe
    df_score=pd.DataFrame(f1_score_data,index=['LogisticReg','MultinomialNB','SVC','RandomForestClassifier'])
    
    return df_score

In [254]:
tax_df_cv=model(df_tax,'Outcome')
tax_df_cv

,F1 Score
LogisticReg,0.500000
MultinomialNB,0.666667
SVC,0.400000
RandomForestClassifier,0.222222


In [256]:
X=df_tax['Case keywords']+df_tax['Number of evidences']
Y=df_tax['Outcome']

X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=50)
v=CountVectorizer()
X_train_v=v.fit_transform(X_train)
X_test_v=v.transform(X_test)
tax_mb=MultinomialNB()
tax_mb.fit(X_train_v,Y_train)

MultinomialNB()

In [264]:
X_test.tail()

25    central excise classification of goods soft se...
56      sales tax turnover tax agricultural producehigh
8      permanent establishment royalty tax deductionLow
94    sale of goods act banking regulation act sales...
44     cgst act sgst act article  alternative remedyLow
dtype: object

In [266]:
cmt1=["Income Tax Act Charitable Purpose Public Utility Profit Motive Medium"]
cmt1_v=v.transform(cmt1)
tax_mb.predict_proba(cmt1_v)[:,1]

array([0.98926264])